# Read wikipedia page for Toronto neighborhood data and perform webscraping to extract the postal data

In [63]:
!pip install BeautifulSoup4
import urllib.request
from bs4 import BeautifulSoup
import csv
urlpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
print(urlpage)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


### Using urllib requests API read the webpage and extract the table and read the results

In [4]:
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')
# find results within table
table = soup.find('table', attrs={'class': 'wikitable sortable'})
results = table.find_all('tr')
print('Number of results', len(results))

Number of results 290


### Main logic is part of the below cell where we read through the cells and extract the data and store them in a data frame.

### we use df.ne to exclude the Borough that is 'Not assigned'.

### we use group by and aggregate with join to group by postal code and add comma seperated values

In [65]:
postal_data = []
COLUMNS = ['PostalCode', 'Borough', 'Neighbourhood']
for row in results:
    cells = row.findAll('td')
    if len(cells) == 0: 
        continue
    postal_data.append([cell.text.strip('\n') for cell in cells])
    
df = pd.DataFrame(postal_data, columns=COLUMNS).reset_index(drop=True)
df = df.loc[df.Borough.ne('Not assigned')].reset_index(drop=True)
#df_new = df.groupby(df['PostalCode']).agg(lambda x: ', '.join(set(x.dropna()))).reset_index(drop=True)
df_new = df.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna()))).reset_index(drop=True)
df_new.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [66]:
df_new.shape

(103, 3)